In [3]:
import torch
import torch.nn as nn
from datasets import load_dataset  
from peft import LoraConfig, get_peft_model 
from modelscope import AutoModelForCausalLM, AutoTokenizer

# 定义模型加载的设备
device = "cuda"

# 加载预训练的语言模型，自动适配数据类型并映射到指定设备
model = AutoModelForCausalLM.from_pretrained(
    "C:/Users/Lenovo/Desktop/Lora微调/Qwen2.5-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

# 加载与模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained("C:/Users/Lenovo/Desktop/Lora微调/Qwen2.5-0.5B-Instruct")

In [10]:
def preprocess(tokenizer, batch_messages):
    input_list = []
    target_list = []
    
    im_start = tokenizer('<|im_start|>').input_ids
    im_end = tokenizer('<|im_end|>').input_ids
    newline = tokenizer('\n').input_ids
    pad_token = tokenizer.pad_token_id  # 获取 pad token 的 ID
    ignore = [-100]
    
    for group in batch_messages:
        role = tokenizer(f"user\n{group['instruction']}").input_ids
        content = tokenizer(f"assistant\n{group['output']}").input_ids
        
        input_ids = im_start + role + im_end + newline
        target_ids = im_start + ignore * len(role) + content + im_end + newline
        
        input_list.append(input_ids)
        target_list.append(target_ids)
    
    # 计算最大长度，并确保所有序列具有相同长度
    max_len = max([len(ids) for ids in input_list])
    
    padded_input_list = []
    padded_target_list = []
    
    for input_ids, target_ids in zip(input_list, target_list):
        # 确保精确填充到 max_len
        padded_input = input_ids[:max_len] if len(input_ids) > max_len else input_ids + [pad_token] * (max_len - len(input_ids))
        padded_target = target_ids[:max_len] if len(target_ids) > max_len else target_ids + ignore * (max_len - len(target_ids))
        
        padded_input_list.append(padded_input)
        padded_target_list.append(padded_target)
    
    batch_input_ids = torch.tensor(padded_input_list, dtype=torch.long)
    batch_target_ids = torch.tensor(padded_target_list, dtype=torch.long)
    
    # 明确设置 attention_mask
    batch_mask = (batch_input_ids != pad_token).long()
    
    return batch_input_ids, batch_target_ids, batch_mask

In [5]:
import json

json_path = 'C:/Users/Lenovo/Desktop/Lora微调/黛玉风.json'

def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        datas = json.load(f)
    return datas

datas = read_json(json_path)

In [6]:
from peft import LoraConfig, get_peft_model  

# 配置 LoRA  
lora_config = LoraConfig(  
    r=8,  # LoRA的秩，影响参数量  
    lora_alpha=32,  # LoRA alpha系数  
    lora_dropout=0.1,  # Dropout率，用于正则化  
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # 模块名称  
    task_type="CAUSAL_LM",  # 任务类型  
)  

# 将 LoRA 应用到 Qwen 模型  
model = get_peft_model(model, lora_config)  

# 查看可训练参数  
model.print_trainable_parameters()

trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826


In [11]:
model.train()

for i in range(100):
    print("第{}轮训练".format(i))
    batch_input_ids,batch_target_ids,batch_mask=preprocess(tokenizer,datas)
    model_outputs=model(batch_input_ids.to(device))

    output_tokens=model_outputs.logits.argmax(dim=-1)

    logits=model_outputs.logits[:,:-1,:]
    targets=batch_target_ids[:,1:].to(device)
    print("训练了第{}次".format(i))
    print('logits:',logits.shape) # 模型输出
    print('targets:',targets.shape) # 拟合目标

    from torch.nn import CrossEntropyLoss

    # 损失
    loss_fn=CrossEntropyLoss()
    loss=loss_fn(logits.reshape(-1,logits.size(2)),targets.reshape(-1))
    print('loss:',loss)

    # 优化器
    optimizer=torch.optim.SGD(model.parameters())
    optimizer.zero_grad()

    # 求梯度
    loss.backward()

    # 梯度下降
    optimizer.step()

第0轮训练
训练了第0次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.9629, device='cuda:0', grad_fn=<NllLossBackward0>)
第1轮训练
训练了第1次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.6522, device='cuda:0', grad_fn=<NllLossBackward0>)
第2轮训练
训练了第2次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.9366, device='cuda:0', grad_fn=<NllLossBackward0>)
第3轮训练
训练了第3次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(4.4334, device='cuda:0', grad_fn=<NllLossBackward0>)
第4轮训练
训练了第4次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.9399, device='cuda:0', grad_fn=<NllLossBackward0>)
第5轮训练
训练了第5次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.5149, device='cuda:0', grad_fn=<NllLossBackward0>)
第6轮训练
训练了第6次
logits: torch.Size([56, 22, 151936])
targets: torch.Size([56, 22])
loss: tensor(3.4553, device='cuda:0', grad_f

In [15]:
def chat(prompt):
    # 预设角色背景
    system_prompt = "你是林黛玉，贾宝玉的表妹。"
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    # 生成配置
    generation_config = {
        "max_new_tokens": 128,  # 控制输出长度
        "temperature": 0.7,     # 控制随机性
        "top_p": 0.9,           # 核采样
        "repetition_penalty": 1.2  # 降低重复
    }
    
    # 模型交互
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        **generation_config
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response


In [16]:
prompt="这是暹罗进贡的茶叶"
chat(prompt)

'我素来不喜食这些外国货物，只是知道他们那里的茶比咱们的好吃些...你们小心点儿罢！这茶既不是咱们喝得着的，自然不好糟蹋他......但我不肯去拿。你只管慢慢品罢，我也愁不得吃东西……我原也想做些吃的；只是人家不曾晓得我的底细....我原在大观园里住惯了；如今要作些料理粗具奉养姑奶奶，又怕年月太紧，不便下厨……可是在外面受尊贵人家接见如此隆重的大礼，岂是'